In [1]:
#!pip install timm
# !pip install opencv-python

In [34]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision
from torchvision import transforms
# import timm
from timm import create_model

import cv2
import numpy as np
import pandas as pd
import os
import datetime
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from torch import nn

### Dataset
Repurposed code from MobileNet model notebook. Changes made include directory reference to content folder, and the keep.csv is already filterd to the classes we wanted to keep.

In [2]:
# Class list
# uploaded class_list to colab session -- todo: revise for local usage
class_list_path = r"C:\Users\linay\Downloads\ifood-2019-fgvc6\class_list.txt"

class_mapping = {}

# Open the file and read the contents
with open(class_list_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        # Split each line by space to separate the index and the class name
        index, class_name = line.strip().split(' ', 1)
        # Convert the index to an integer and add the mapping to the dictionary
        class_mapping[int(index)] = class_name
    print(class_mapping.values())

dict_values(['macaron', 'beignet', 'cruller', 'cockle_food', 'samosa', 'tiramisu', 'tostada', 'moussaka', 'dumpling', 'sashimi', 'knish', 'croquette', 'couscous', 'porridge', 'stuffed_cabbage', 'seaweed_salad', 'chow_mein', 'rigatoni', 'beef_tartare', 'cannoli', 'foie_gras', 'cupcake', 'osso_buco', 'pad_thai', 'poutine', 'ramen', 'pulled_pork_sandwich', 'bibimbap', 'chicken_kiev', 'apple_pie', 'risotto', 'fruitcake', 'chop_suey', 'haggis', 'scrambled_eggs', 'frittata', 'scampi', 'sushi', 'orzo', 'fritter', 'nacho', 'beef_stroganoff', 'beef_wellington', 'spring_roll', 'savarin', 'crayfish_food', 'souffle', 'adobo', 'streusel', 'deviled_egg', 'escargot', 'club_sandwich', 'carrot_cake', 'falafel', 'farfalle', 'terrine', 'poached_egg', 'gnocchi', 'bubble_and_squeak', 'egg_roll', 'caprese_salad', 'sauerkraut', 'creme_brulee', 'pavlova', 'fondue', 'scallop', 'jambalaya', 'tempura', 'chocolate_cake', 'potpie', 'spaghetti_bolognese', 'sukiyaki', 'applesauce', 'baklava', 'salisbury_steak', 'lin

In [3]:
# Reduce classes after EDA process
keep_list = r"C:\Users\linay\Downloads\ifood-2019-fgvc6\keep.csv"
df_keep = pd.read_csv(keep_list)
## commenting out the below code because I already filtered the csv file to all of the classes we wanted to keep
# df_keep = df_keep[df_keep['All Agree']=='True']
print("Classes to train for :", len(df_keep))
classes_to_keep = df_keep['category_name'].tolist()

Classes to train for : 25


In [4]:
# Convert to an array of label IDs for classes to keep
filtered_class_mapping = {index: class_name for index, class_name in class_mapping.items() if class_name in classes_to_keep}
print("Filtered class mapping:", len(filtered_class_mapping))
filtered_class_ids = list(map(str, filtered_class_mapping.keys()))

Filtered class mapping: 25


In [5]:
# Check "bad" input test data, and return only those that are good
type_1_data = pd.read_csv(r"C:\Users\linay\Downloads\ifood-2019-fgvc6\df_eda.csv")
type_1_data = type_1_data[(type_1_data['is_cartoon'] == False) & (type_1_data['has_face'] == False) & (type_1_data['is_media_art'] == False) & (type_1_data['has_text'] == False)]
type_1_data = type_1_data['img_name'].to_list()
print("count of good data:", len(type_1_data))

count of good data: 40631


In [6]:
# Paths to the dataset
train_dir = r"C:\Users\linay\Downloads\ifood-2019-fgvc6\train_set\train_set"
val_dir   = r"C:\Users\linay\Downloads\ifood-2019-fgvc6\val_set\val_set"
test_dir  = r"C:\Users\linay\Downloads\ifood-2019-fgvc6\test_set\test_set"

# Path to the labels
df_train = pd.read_csv(r"C:\Users\linay\Downloads\ifood-2019-fgvc6\train_labels.csv")
df_val   = pd.read_csv(r"C:\Users\linay\Downloads\ifood-2019-fgvc6\val_labels.csv")

# TypeError: If class_mode="categorical", y_col="label" column values must be type string, list or tuple.
df_train['label'] = df_train['label'].astype(str)
df_val['label']   = df_val['label'].astype(str)

# Filter datasets to just classifications of interest
df_train = df_train[df_train['label'].isin(filtered_class_ids)]
print("This is the number of images that are in the filtered class in train set:", len(df_train))
df_train = df_train[df_train['img_name'].isin(type_1_data)]
print("This is the number of images that are in the the filtered class and also are not bad images in train set:", len(df_train))
df_val   = df_val[df_val['label'].isin(filtered_class_ids)]
print("This is the number of images that are in the validation set that are in the filtered class:", len(df_val))

This is the number of images that are in the filtered class in train set: 13578
This is the number of images that are in the the filtered class and also are not bad images in train set: 5596
This is the number of images that are in the validation set that are in the filtered class: 1263


In [7]:
# append a new column and store the image directory in the columns
df_train['image_directory'] = ''
for index, row in df_train.iterrows():
    df_train.loc[index, 'image_directory'] = os.path.join(train_dir, row['img_name']) 

# change the labels to integer 
df_train['label'] = df_train['label'].astype(int)

# append a new column and store the image directory in the colums 
df_val['image_directory'] = ''
for index, row in df_val.iterrows():
    df_val.loc[index, 'image_directory'] = os.path.join(val_dir, row['img_name']) 

df_val['label'] = df_val['label'].astype(int)

In [9]:
print("This is the number of unique labels in the validation set:", len(pd.unique(df_val['label'])))

This is the number of unique labels in the validation set: 25


In [19]:
# df_val.to_csv('df_val_check.csv', index=False)

In [11]:
# # CUTE DELETE LATER
# # Remove the images that we want to keep for classification 
# # Create a new folder to store the filtered dataset 
# # Identify the images for just classification interest 
# image_names = df_train['img_name'].tolist()
# print("number of images for training", len(image_names)) 
# # instantiate an empty list to store matched images 
# matched_image_directory = []

# # iterate through the file names and check if each image exists in the directory 
# count = 0
# for image in image_names:
#     curr_train_dir = os.path.join(train_dir, image)
#     # testing current image train_dir (setting curr_train_dir for EACH image, using global train_dir variable)
#     # if (count < 5):
#     #     print(curr_train_dir)
#     #     count += 1
#     if os.path.exists(train_dir):
#         matched_image_directory.append(train_dir)
# # print or use the list of matched images
# print("number of images in the directory:", len(matched_image_directory))

In [12]:
# # CUTE DELETE LATER
# create the destination directory to store all of the filtered,clean images
# new_train_dir = r"C:\Users\linay\Downloads\ifood-2019-fgvc6\train_set\train_set_filtered"
# if not os.path.exists(new_train_dir):
#     os.makedirs(new_train_dir)
    
# if os.path.exists(new_train_dir):
#     for filename in os.listdir(new_train_dir): 
#         file_path = os.path.join(new_train_dir, filename)
#         if os.path.isfile(file_path):
#             os.remove(file_path)
#             # print(f"Deleted file: {file_path}")

# for image in image_names:
#     src_file_path = os.path.join(train_dir, image)
#     # print(src_file_path)
#     if os.path.exists(src_file_path):
#         shutil.copy2(src_file_path, new_train_dir)
#         # print(f"Copied {file_name} to {new_train_dir}")

### Create and Evaluate Model

In [60]:
# Load training dataset and create data loader

# takes the df_train pandas dataframe 
# and transforms as inputs and returns the image along with corresponding label
class CustomImageDataset(Dataset): 
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_path = self.df.iloc[index, 2]  # assuming the image path is in the 3 column
        label = self.df.iloc[index, 1]  # assuming the label is in the second column
        image = Image.open(image_path)
        image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# # standard transformation
# transform = transforms.Compose([
#     transforms.Resize(224),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# applying data augmentation
transform = transforms.Compose([
    transforms.Resize(256),  # Resize to 256x256
    transforms.RandomCrop(224),  # Randomly crop to 224x224
    transforms.RandomRotation(30),  # Randomly rotate up to 30 degrees
    transforms.RandomHorizontalFlip(),  # Randomly flip horizontally
    transforms.RandomAffine(degrees=30, scale=(0.8, 1.2)),  # Randomly scale and rotate
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # Randomly jitter color
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# create training data set and data loader
data_set = CustomImageDataset(df_train, transform)
data_loader = DataLoader(data_set, batch_size=32, shuffle=True)

# create validation dataset and data loader
validation_data_set = CustomImageDataset(df_val, transform)
validation_data_loader = DataLoader(validation_data_set, batch_size=32, shuffle=False)

print('Data Transformed at:', datetime.datetime.now())

Data Transformed at: 2024-10-13 22:22:59.888046


In [62]:
print('Start Time:', datetime.datetime.now())

# Load pre-trained DeiT model
model = create_model('deit_base_patch16_224', pretrained=True)
# model.dropout = nn.Dropout(p=0.2)

# Train the model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

# Instantiate lists to store training and validation loss and accuracy
train_loss = []
train_accuracy = []
train_accuracy_top1 = []
train_accuracy_top5 = []
val_loss = []
val_accuracy = []
val_accuracy_top1 = []
val_accuracy_top5 = []
lr_pg0 = []
lr_pg1 = []
lr_pg2 = []
epochs = []

# create a function to evaluate the model on the validation set 
def evaluate(model, data_loader):
    model.eval()
    total_loss = 0
    total_correct = 0 
    total_correct_top1 = 0
    total_correct_top5 = 0
    confidence_values = [] 
    predicted_labels = []
    image_paths = []
    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted_top1 = torch.max(outputs, 1)
            _, predicted_top5 = torch.topk(outputs, k=5, dim=1)
            total_correct_top1 += (predicted_top1 == labels).sum().item()
            total_correct_top5 += (labels.unsqueeze(1) == predicted_top5).any(dim=1).sum().item()
            probs = torch.nn.functional.softmax(outputs, dim=1)
            confidence_values.extend(probs[range(len(labels)), labels].cpu().numpy())
            predicted_labels.extend(predicted_top1.cpu().numpy())  # <--- Use predicted_top1 instead of labels
            image_paths.extend([data_loader.dataset.df.iloc[i, 2] for i in range(len(labels))])         
    accuracy_top1 = total_correct_top1 / len(data_loader.dataset)
    accuracy_top5 = total_correct_top5 / len(data_loader.dataset)
    return total_loss / len(data_loader), accuracy_top1, accuracy_top5, confidence_values, predicted_labels, image_paths

# instantiate the patience count and minimum delta
patience = 7
min_delta = 0.0005

best_val_accuracy = 0
patience_count = 0

for epoch in range(50):
    total_loss = 0 
    total_correct_top1 = 0
    total_correct_top5 = 0
    for images, labels in data_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels.squeeze())

        # Backward pass
        loss.backward()

        # Update model parameters
        optimizer.step()

        total_loss += loss.item()
        _, predicted_top1 = torch.max(outputs, 1)
        _, predicted_top5 = torch.topk(outputs, k=5, dim=1)
        total_correct_top1 += (predicted_top1 == labels).sum().item()
        total_correct_top5 += (labels.unsqueeze(1) == predicted_top5).any(dim=1).sum().item()
    
    # Calculate training loss and accuracy
    train_loss.append(total_loss / len(data_loader))
    train_accuracy_top1.append(total_correct_top1 / len(data_loader.dataset))
    train_accuracy_top5.append(total_correct_top5 / len(data_loader.dataset))
    lr_pg0.append(optimizer.param_groups[0]['lr'])
    lr_pg1.append(optimizer.param_groups[0]['lr'])
    lr_pg2.append(optimizer.param_groups[0]['lr'])
    epochs.append(epoch+1)

    # evaluate the model on the validation set
    model.eval()
    validation_loss, validation_accuracy_top1, validation_accuracy_top5, confidence_values, labels, image_paths = evaluate(model, validation_data_loader)
    val_loss.append(validation_loss)
    val_accuracy_top1.append(validation_accuracy_top1)
    val_accuracy_top5.append(validation_accuracy_top5)
    
    # check to see if validation loss has improved 
    # stop if plateaus 
    if validation_accuracy_top5 < best_val_accuracy - min_delta:
        best_val_accuracy = validation_accuracy_top5
        patience_count = 0
    else:
        patience_count += 1

    if patience_count >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break    

    # print loss and accuracy for each epoch 
    print('Epoch {}: Loss = {:.4f}, Validation Loss = {:.4f}, Validation Accuracy Top1 = {:.4f}, Validation Accuracy Top5 = {:.4f}'.format(epoch+1, loss.item(), validation_loss, validation_accuracy_top1, validation_accuracy_top5))

print('Epoch End Time:', datetime.datetime.now())

# # create a dataframe to store images, predicted labels, and confidence values
confidence_values = []
image_paths = []
predicted_labels = []
batch_indices = []
with torch.no_grad():
    for i, (images, labels) in enumerate(validation_data_loader):
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        confidence_values.extend(probs[range(len(labels)), labels].cpu().numpy())
        batch_indices.extend(validation_data_loader.dataset.df.index[i*validation_data_loader.batch_size:(i+1)*validation_data_loader.batch_size])
        image_paths.extend(validation_data_loader.dataset.df.loc[batch_indices[-len(labels):], 'image_directory'])
        predicted_labels.extend(predicted.cpu().numpy())

confidence_values_df = pd.DataFrame({'image_directory': image_paths, 'label': predicted_labels, 'confidence': confidence_values})
confidence_values_df.to_csv('confidence_values.csv', index=False)

print('Validation Results End Time:', datetime.datetime.now())

## Store Training Log as a CSV
# Create a dictionary to store the training log csv
data = {
    'Epoch': epochs,
    'Train/Loss': train_loss,
    'Train_Metrics/Accuracy_Top1': train_accuracy_top1,
    'Train_Metrics/Accuracy_Top5': train_accuracy_top5,
    'Val/Loss': val_loss,
    'Val_Metrics/Accuracy_Top1': val_accuracy_top1,
    'Val_Metrics/Accuracy_Top5': val_accuracy_top5,
    'LR/Pg0': lr_pg0,
    'LR/Pg1': lr_pg1,
    'LR/Pg2': lr_pg2,
} 

# Convert the dictionary to a Pandas DataFrame
training_log_df = pd.DataFrame(data)

# Store the DataFrame to a CSV file
training_log_df.to_csv('deit_training_log.csv', index=False)

## Store Validation Results as a CSV
# Read in class list text file into df
class_df = pd.read_csv('class_list.txt', sep=' ', names=['label', 'class'], index_col=None)

# append true labels from validation df 
validation_results_df = pd.merge(confidence_values_df, df_val, on='image_directory')

# rename labels to true and predicted
validation_results_df = validation_results_df.rename(columns={
    'label_x': 'predicted_labels',
    'label_y': 'true_labels'
})

# append class for true labels
validation_results_df['true_class'] = validation_results_df['true_labels'].map(class_df.set_index('label')['class'])

# append class for predicted labels 
validation_results_df['predicted_class'] = validation_results_df['predicted_labels'].map(class_df.set_index('label')['class'])

# reorder columns 
validation_results_df = validation_results_df.reindex(columns=['image_directory', 'img_name', 'true_labels', 'true_class', 'predicted_labels', 'predicted_class', 'confidence'])

validation_results_df.to_csv('validation_results.csv', index=False)

Start Time: 2024-10-13 22:28:41.807947
Epoch 1: Loss = 2.0761, Validation Loss = 1.8783, Validation Accuracy Top1 = 0.4473, Validation Accuracy Top5 = 0.8052
Epoch 2: Loss = 1.2718, Validation Loss = 1.3513, Validation Accuracy Top1 = 0.5914, Validation Accuracy Top5 = 0.9002
Epoch 3: Loss = 0.9327, Validation Loss = 1.1474, Validation Accuracy Top1 = 0.6580, Validation Accuracy Top5 = 0.9161
Epoch 4: Loss = 0.7109, Validation Loss = 1.2349, Validation Accuracy Top1 = 0.6334, Validation Accuracy Top5 = 0.9105
Epoch 5: Loss = 0.8850, Validation Loss = 1.0774, Validation Accuracy Top1 = 0.6746, Validation Accuracy Top5 = 0.9272
Epoch 6: Loss = 1.2863, Validation Loss = 1.1172, Validation Accuracy Top1 = 0.6611, Validation Accuracy Top5 = 0.9264
Early stopping at epoch 7
Epoch End Time: 2024-10-14 01:35:27.204442
Validation Results End Time: 2024-10-14 01:37:19.846872


In [64]:
# Summary Metrics
accuracy = accuracy_score(validation_results_df['true_labels'], validation_results_df['predicted_labels'])
precision = precision_score(validation_results_df['true_labels'], validation_results_df['predicted_labels'], average='macro')
recall = recall_score(validation_results_df['true_labels'], validation_results_df['predicted_labels'], average='macro', zero_division=0)
f1 = f1_score(validation_results_df['true_labels'], validation_results_df['predicted_labels'], average='macro', zero_division=0)

# auc roc score
lb = LabelBinarizer()
y_true = lb.fit_transform(validation_results_df['true_labels'])
y_pred_proba = np.array([validation_results_df['confidence']]*y_true.shape[1]).T

# Calculate the weighted AUC-ROC score
auc_roc_weighted = roc_auc_score(y_true, y_pred_proba, average='weighted')

# # Calculate the roc_auc_score for each class
# for i in range(y_true.shape[1]):
#     auc_roc = roc_auc_score(y_true[:, i], validation_results_df['confidence'])
#     print(f"AUC-ROC for class {i}: {auc_roc}")

# Get the top 3 predicted labels
top_3_predicted_labels = []
for i in range(len(validation_results_df['confidence'])):
    top_3_indices = np.argsort(validation_results_df['confidence'].iloc[i])[-3:][::-1]
    top_3_predicted_labels.append(np.array([validation_results_df['predicted_labels'].iloc[i]]))

# Check if true label is in top 3 predicted labels
correct_top_3 = [x in y for x, y in zip(validation_results_df['true_labels'], top_3_predicted_labels)]

# Calculate the top 3 accuracy
top_3_accuracy = np.mean(correct_top_3)

# print(f'Top 3 Accuracy: {top_3_accuracy}')
print(f"Weighted AUC-ROC score: {auc_roc_weighted}")
print(accuracy, precision, recall, f1)
print(f"Top 3 accuracy: {top_3_accuracy}")

Weighted AUC-ROC score: 0.500264838031961
0.6690419635787807 0.6997493566728048 0.6620434517703027 0.6605874531959894
Top 3 accuracy: 0.6690419635787807
